# Forecasting Freshwater Algal Bloom Levels Using Multisource Climate and Water-Quality Data

*This is the course project of **STATS 402: Interdisciplinary Data Analysis**.*

**Name:** Ziyue Yin

**NetID:** zy166

## Dataset: HydroLAKES

Lake polygons (including all attributes) in shapefileformat: https://www.hydrosheds.org/products/hydrolakes.

## Dataset: NASA OceanColor Inland Waters (ILW)

S3Merged-ILW data: https://oceandata.sci.gsfc.nasa.gov/directdataaccess/Level-3%20Mapped/Merged-S3-ILW/.

S3B-ILW data: https://oceandata.sci.gsfc.nasa.gov/directdataaccess/Level-3%20Mapped/S3B-ILW/.

After downloading the datasets, the structure should be shown as follows:

```
datasets/
 ├── ILW/
 │    ├── S3B/2024/CONUS_MO/
 │    │      ├── S3B_OLCI_EFRNT.20240101_20240131.L3m.MO.ILW_CONUS.V5.all.CONUS.300m.nc
 │    │      ├── S3B_OLCI_EFRNT.20240201_20240229.L3m.MO.ILW_CONUS.V5.all.CONUS.300m.nc
 │    ├── Merged/2024/CONUS_DAY/
 │    │      ├── S3M_OLCI_EFRNT.20240101.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc
 │    │      ├── S3M_OLCI_EFRNT.20240102.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc
 │    │      ...
```

### Data Structure Exploration

First of all, let's glance at the monthly dataset.

In [1]:
import xarray as xr

p = "/dkucc/home/zy166/HAB-forecasting/datasets/ILW/S3B/2024/CONUS_MO/S3B_OLCI_EFRNT.20240101_20240131.L3m.MO.ILW_CONUS.V5.all.CONUS.300m.nc"

ds = xr.open_dataset(p, engine="netcdf4", chunks="auto")
print(ds.dims)
print(list(ds.data_vars))

FrozenMappingWarningOnValuesAccess({'y': 15138, 'x': 26328, 'rgb': 3, 'eightbitcolor': 256})
['rhos_400', 'rhos_412', 'rhos_443', 'rhos_490', 'rhos_510', 'rhos_560', 'rhos_620', 'rhos_665', 'rhos_674', 'rhos_681', 'rhos_709', 'rhos_754', 'rhos_865', 'rhos_884', 'CI_cyano', 'palette']


And also, the daily dataset.

In [2]:
import xarray as xr

p = "/dkucc/home/zy166/HAB-forecasting/datasets/ILW/Merged/2024/CONUS_DAY/S3M_OLCI_EFRNT.20240101.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc"

ds = xr.open_dataset(p, engine="netcdf4", chunks="auto")
print(ds.dims)
print(list(ds.data_vars))

FrozenMappingWarningOnValuesAccess({'y': 15138, 'x': 26328, 'rgb': 3, 'eightbitcolor': 256})
['rhos_400', 'rhos_412', 'rhos_443', 'rhos_490', 'rhos_510', 'rhos_560', 'rhos_620', 'rhos_665', 'rhos_674', 'rhos_681', 'rhos_709', 'rhos_754', 'rhos_865', 'rhos_884', 'CI_cyano', 'palette']


### Target Feature Extraction

#### Utilities

In [1]:
import re
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import rioxarray

def infer_time_label(nc_path, ds, product="monthly"):
    """
    Return a pandas.Timestamp, try to infer from ds or filename.
    product: 'monthly' or 'daily'
    """
    # 1) Directly have time coordinate/variable
    if "time" in ds.coords or "time" in ds.variables:
        try:
            tt = pd.to_datetime(ds["time"].values)
            tt = np.array(tt).reshape(-1)[0]
            return pd.to_datetime(tt)
        except Exception:
            pass

    # 2) Global attributes (common in L3M data)
    start = ds.attrs.get("time_coverage_start") or ds.attrs.get("start_time")
    end   = ds.attrs.get("time_coverage_end")   or ds.attrs.get("end_time")
    if start and end:
        try:
            ts = pd.to_datetime(start)
            te = pd.to_datetime(end)
            if product == "monthly":
                return ts + (te - ts) / 2
            else:
                return ts
        except Exception:
            pass

    # 3) Analyze the filename
    fn = nc_path.split("/")[-1]
    if product == "monthly":
        # ...YYYYMMDD_YYYYMMDD.L3m.MO...
        m = re.search(r"\.(\d{8})_(\d{8})\.L3m\.MO\.", fn)
        if m:
            b, e = m.group(1), m.group(2)
            ts = pd.to_datetime(b, format="%Y%m%d")
            te = pd.to_datetime(e, format="%Y%m%d")
            return ts + (te - ts) / 2
    else:
        # ...YYYYMMDD.L3m.DAY...
        m = re.search(r"\.(\d{8})\.L3m\.DAY\.", fn)
        if m:
            return pd.to_datetime(m.group(1), format="%Y%m%d")

    raise ValueError("Cannot infer time from dataset or filename: " + fn)

def clean_ci(da: xr.DataArray) -> xr.DataArray:
    """
    Filter out values out of physical range and remove near-zero values.
    """
    vmin = float(da.attrs.get("valid_min", np.nan))
    vmax = float(da.attrs.get("valid_max", np.nan))
    if np.isfinite(vmin):
        da = da.where(da >= vmin)
    if np.isfinite(vmax):
        da = da.where(da <= vmax)

    thr = max(vmin, 5e-5) if np.isfinite(vmin) else 5e-5
    da = da.where(da > thr)

    return da

def extract_lakes_from_nc(nc_path: str,
                          lakes_gdf: gpd.GeoDataFrame,
                          lake_id_col: str,
                          product: str) -> pd.DataFrame:
    """
    nc_path: A single NetCDF file (S3B monthly or S3M daily)
    lakes_gdf: A GeoDataFrame containing `lake_id` and `geometry` (EPSG:4326)
    product: 'monthly' | 'daily'
    Returns: One row per lake (timestamp of the file)
    """
    with xr.open_dataset(nc_path, engine="netcdf4", chunks="auto") as ds:
        t  = infer_time_label(nc_path, ds, product=product)
        da = set_spatial_dims_safe(ds["CI_cyano"], ds=ds)
        da = clean_ci(da).rio.write_crs(4326)

        rows = []
        for _, row in lakes_gdf.iterrows():
            lid  = row[lake_id_col]
            geom = [row.geometry]
            try:
                clipped = da.rio.clip(geom, lakes_gdf.crs, drop=True)
                arr     = clipped.data  # dask/np 数组
                arr     = np.asarray(arr)  # 触发 dask 计算
                mask    = np.isfinite(arr)
                n_valid = int(mask.sum())

                if n_valid == 0:
                    mean_val = np.nan
                    p90      = np.nan
                else:
                    vals = arr[mask].ravel()
                    mean_val = float(np.nanmean(vals))
                    p90      = float(np.nanquantile(vals, 0.9))
            except Exception:
                mean_val, p90, n_valid = np.nan, np.nan, 0

            rows.append({
                "lake_id": lid, "time": pd.to_datetime(t), "product": product,
                "CI_mean": mean_val, "CI_p90": p90, "n_valid": n_valid,
                "src": Path(nc_path).name,
            })
    return pd.DataFrame(rows)

def set_spatial_dims_safe(da: xr.DataArray, ds: xr.Dataset | None = None) -> xr.DataArray:
    """
    Make `da` geospatially aware for rioxarray:
    - Prefer real dimension names present on `da` (lon/lat or x/y or variants);
    - Only pass EXISTING dimension names to `rio.set_spatial_dims`;
    - If dataset carries 1D lon/lat arrays matching x/y lengths, bind them as coords;
    - Finally write CRS=EPSG:4326.

    NOTE: If lon/lat are 2D (curvilinear), we DO NOT pass them as dims; we keep x/y dims.
    """
    dims = list(da.dims)

    # normalize common variants
    def _has(*cands):
        return any(c in dims for c in cands)

    # Case A: dims already lon/lat
    if "lon" in dims and "lat" in dims:
        out = da.rio.write_crs(4326)
        out = out.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=False)
    elif "longitude" in dims and "latitude" in dims:
        out = da.rename({"longitude": "lon", "latitude": "lat"})
        out = out.rio.write_crs(4326)
        out = out.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=False)

    # Case B: dims are x/y (any case)
    elif (_has("x") and _has("y")) or (_has("X") and _has("Y")):
        # rename upper-case to lower-case to please rioxarray
        rename_map = {}
        if "X" in dims: rename_map["X"] = "x"
        if "Y" in dims: rename_map["Y"] = "y"
        out = da.rename(rename_map) if rename_map else da
        out = out.rio.write_crs(4326)
        out = out.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)

        # try binding 1D lon/lat coords from ds if lengths match
        if ds is not None:
            try:
                lon1d = None
                lat1d = None
                for lon_name in ("lon","longitude"):
                    if lon_name in ds.variables and ds[lon_name].ndim == 1 and ds[lon_name].sizes[ds[lon_name].dims[0]] == out.sizes["x"]:
                        lon1d = np.asarray(ds[lon_name].values)
                        break
                for lat_name in ("lat","latitude"):
                    if lat_name in ds.variables and ds[lat_name].ndim == 1 and ds[lat_name].sizes[ds[lat_name].dims[0]] == out.sizes["y"]:
                        lat1d = np.asarray(ds[lat_name].values)
                        break
                if lon1d is not None and lat1d is not None:
                    out = out.assign_coords(x=lon1d, y=lat1d)
            except Exception:
                pass

    # Case C: unknown names → rename the last two dimensions to y/x
    elif len(dims) >= 2:
        ydim, xdim = dims[-2], dims[-1]
        out = da.rename({xdim: "x", ydim: "y"}).rio.write_crs(4326)
        out = out.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)

    else:
        raise ValueError(f"Cannot determine spatial dims for {da.name!r}; dims={dims}")

    return out

def looks_like_hdf5(path: Path) -> bool:
    try:
        with open(path, "rb") as f:
            sig = f.read(8)
        return sig.startswith(b"\x89HDF") or sig.startswith(b"CDF")
    except Exception:
        return False

def _extract_one_with_h5netcdf(nc_path: Path,
                               lakes_gdf: gpd.GeoDataFrame,
                               lake_id_col: str,
                               product: str = "daily") -> pd.DataFrame:
    """
    兜底方案：用 h5netcdf 打开并在本函数内完成裁剪与统计。
    只有当 extract_lakes_from_nc 失败时才会被调用。
    """
    rows = []
    # 注意：phony_dims="access" 让 h5netcdf 创建可访问维度名
    with xr.open_dataset(nc_path, engine="h5netcdf", chunks="auto", phony_dims="access") as ds:
        t  = infer_time_label(str(nc_path), ds, product=product)
        da = set_spatial_dims_safe(ds["CI_cyano"], ds=ds)
        da = clean_ci(da).rio.write_crs(4326)

        for _, r in lakes_gdf.iterrows():
            lid  = r[lake_id_col]
            geom = [r.geometry]
            try:
                clipped = da.rio.clip(geom, lakes_gdf.crs, drop=True)
                arr     = np.asarray(clipped.data)
                mask    = np.isfinite(arr)
                n_valid = int(mask.sum())
                if n_valid == 0:
                    mean_val, p90 = np.nan, np.nan
                else:
                    vals = arr[mask].ravel()
                    mean_val = float(np.nanmean(vals))
                    p90      = float(np.nanquantile(vals, 0.9))
            except Exception:
                mean_val, p90, n_valid = np.nan, np.nan, 0

            rows.append({
                "lake_id": lid,
                "date":    pd.to_datetime(t),
                "product": product,
                "CI_mean": mean_val,
                "CI_p90":  p90,
                "n_valid": n_valid,
                "src":     nc_path.name,
                "engine":  "h5netcdf",
            })
    return pd.DataFrame(rows)

def try_open_xarray(fp: Path):
    """Try netcdf4 → h5netcdf → h5py，return (ds or None, engine_used)。"""
    try:
        ds = xr.open_dataset(fp, engine="netcdf4", chunks="auto")
        _ = ds.dims
        return ds, "netcdf4"
    except Exception as e1:
        try:
            ds = xr.open_dataset(fp, engine="h5netcdf", chunks="auto", phony_dims="access")
            _ = ds.dims
            return ds, "h5netcdf"
        except Exception as e2:
            try:
                with h5py.File(fp, "r") as f:
                    pass
                print(f"[WARN] {fp.name} readable by h5py but not by xarray engines (netCDF-4 layout issue?)")
            except Exception as e3:
                print(f"[WARN] {fp.name} not readable even by h5py: {e3}")
            print(f"[SKIP] {fp.name} → netcdf4:{e1} | h5netcdf:{e2}")
            return None, None

#### Scale 1: In general

##### Monthly

Here, we use the **S3B Monthly** data. One month per row.

In [4]:
import glob, numpy as np, pandas as pd, xarray as xr
from pathlib import Path

monthly_dir = Path("/dkucc/home/zy166/HAB-forecasting/datasets/ILW/S3B/2024/CONUS_MO")
out_csv = monthly_dir/"ci_cyano_monthly_mean.csv"

rows = []
for fp in sorted(monthly_dir.glob("S3B_OLCI_EFRNT.*.L3m.MO.ILW_CONUS.V5.all.CONUS.300m.nc")):
    with xr.open_dataset(fp, engine="netcdf4", chunks="auto") as ds:
        da = clean_ci(ds["CI_cyano"])
        arr = np.asarray(da.data)
        mask = np.isfinite(arr)
        m   = float(np.nanmean(arr[mask])) if mask.any() else np.nan
        p90 = float(np.nanquantile(arr[mask], 0.9))    if mask.any() else np.nan
        t   = infer_time_label(str(fp), ds, product="monthly")
        rows.append({"time": pd.to_datetime(t), "CI_mean": m, "CI_p90": p90,
                     "n_valid": int(mask.sum())})

df_mo = pd.DataFrame(rows).sort_values("time").reset_index(drop=True)
df_mo.to_csv(out_csv, index=False)
df_mo.head()

,time,CI_mean,CI_p90,n_valid
0,2024-01-16 16:53:28.500000+00:00,0.000591,0.000472,2030281
1,2024-02-15 17:12:11.500000+00:00,0.000539,0.000351,2204191
2,2024-03-16 16:44:09+00:00,0.000492,0.000166,1810493
3,2024-04-16 04:52:02+00:00,0.000496,0.000540,2151149
4,2024-05-16 17:01:29+00:00,0.000500,0.000836,2586974


##### Daily

Here, we use the **S3M Daily** data. One day per row.

In [5]:
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import h5py
import geopandas as gpd

daily_dir = Path("/dkucc/home/zy166/HAB-forecasting/datasets/ILW/Merged/2024/CONUS_DAY")
out_csv = daily_dir / "ci_cyano_daily_mean.csv"

rows = []
for fp in sorted(daily_dir.glob("S3M_OLCI_EFRNT.*.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc")):
    if not looks_like_hdf5(fp):
        print(f"[WARN] Skip (not HDF5 header): {fp.name}")
        continue

    ds, eng = try_open_xarray(fp)
    if ds is None:
        continue

    try:
        da = clean_ci(ds["CI_cyano"])
        arr = np.asarray(da.data)
        mask = np.isfinite(arr)
        m   = float(np.nanmean(arr[mask])) if mask.any() else np.nan
        p90 = float(np.nanquantile(arr[mask], 0.9)) if mask.any() else np.nan

        t = infer_time_label(str(fp), ds, product="daily")

        rows.append({
            "date":   pd.to_datetime(t),
            "CI_mean": m,
            "CI_p90":  p90,
            "n_valid": int(mask.sum()),
            "src":     fp.name,
            "engine":  eng,
        })
    finally:
        ds.close()

df_day = pd.DataFrame(rows).sort_values("date").reset_index(drop=True)
df_day.to_csv(out_csv, index=False)
print(f"[OK] saved → {out_csv} (rows={len(df_day)})")
df_day.head()

[WARN] S3M_OLCI_EFRNT.20240211.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc not readable even by h5py: Unable to synchronously open file (truncated file: eof = 323158016, sblock->base_addr = 0, stored_eof = 763662772)
[SKIP] S3M_OLCI_EFRNT.20240211.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc → netcdf4:[Errno -101] NetCDF: HDF error: '/dkucc/home/zy166/HAB-forecasting/datasets/ILW/Merged/2024/CONUS_DAY/S3M_OLCI_EFRNT.20240211.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc' | h5netcdf:Unable to synchronously open file (truncated file: eof = 323158016, sblock->base_addr = 0, stored_eof = 763662772)
[WARN] S3M_OLCI_EFRNT.20241016.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc not readable even by h5py: Unable to synchronously open file (truncated file: eof = 411697152, sblock->base_addr = 0, stored_eof = 978029795)
[SKIP] S3M_OLCI_EFRNT.20241016.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc → netcdf4:[Errno -101] NetCDF: HDF error: '/dkucc/home/zy166/HAB-forecasting/datasets/ILW/Merged/2024/CONUS_DAY/S3M_OLCI_EFRNT.2024101

,date,CI_mean,CI_p90,n_valid,src,engine
0,2024-01-01 13:52:31+00:00,0.000432,0.00005,1024210,S3M_OLCI_EFRNT.20240101.L3m.DAY.ILW_CONUS.V5.a...,netcdf4
1,2024-01-02 13:36:49+00:00,0.000274,0.00005,1047655,S3M_OLCI_EFRNT.20240102.L3m.DAY.ILW_CONUS.V5.a...,netcdf4
2,2024-01-03 14:02:06+00:00,0.000368,0.00005,865219,S3M_OLCI_EFRNT.20240103.L3m.DAY.ILW_CONUS.V5.a...,netcdf4
3,2024-01-04 13:40:26+00:00,0.000230,0.00005,1144789,S3M_OLCI_EFRNT.20240104.L3m.DAY.ILW_CONUS.V5.a...,netcdf4
4,2024-01-05 13:48:41+00:00,0.000273,0.00005,1152641,S3M_OLCI_EFRNT.20240105.L3m.DAY.ILW_CONUS.V5.a...,netcdf4


#### Scale 2: Five Great Lakes

First of all, we get the five Great Lakes out explicitly.

In [6]:
from pathlib import Path
import geopandas as gpd

src = "/dkucc/home/zy166/HAB-forecasting/datasets/Lakes/shapes/lakes_greatlakes.gpkg"
gdf = gpd.read_file(src)

keep_names = ["Superior","Michigan","Huron","Erie","Ontario"]
gdf = gpd.read_file(src)
gdf5 = gdf[gdf["Lake_name"].str.fullmatch("|".join(keep_names), case=False)].copy()
gdf5 = gdf5.dissolve(by="Lake_name", as_index=False).rename(columns={"Lake_name":"lake_name"})

order = ["Erie","Huron","Michigan","Ontario","Superior"]
gdf5 = gdf5.set_index("lake_name").loc[order].reset_index()

gdf5m = gdf5.to_crs(5070)
gdf5m["geometry"] = gdf5m.buffer(-300)
gdf5m = gdf5m[~gdf5m.geometry.is_empty].copy()
gdf5  = gdf5m.to_crs(4326).reset_index(drop=True)

gdf5["lake_id"] = [f"GL-{i+1}" for i in range(len(gdf5))]

# Buffer the shoreline by 300 m, first to equidistant projection, then buffer, then back to 4326
gdf5m = gdf5.to_crs(5070)
gdf5m["geometry"] = gdf5m.buffer(-300)
gdf5 = gdf5m.to_crs(4326)

out = Path(src).with_name("lakes_greatlakes_5poly.gpkg")
gdf5.to_file(out, driver="GPKG")
print("Saved:", out, "features:", len(gdf5))

Saved: /dkucc/home/zy166/HAB-forecasting/datasets/Lakes/shapes/lakes_greatlakes_5poly.gpkg features: 5


##### Monthly

In [7]:
def run_monthly(monthly_dir: str,
                lakes_fp: str,
                lake_id_col: str,
                out_parquet: str):
    """
    monthly_dir: Directory containing files like S3B_OLCI_EFRNT.*.L3m.MO.ILW_CONUS...nc
    lakes_fp:    Lake boundaries (gpkg/shp, must be EPSG:4326)
    """
    gdf = gpd.read_file(lakes_fp)
    if gdf.crs is None:
        raise ValueError("The lake file is missing CRS, please ensure it is EPSG:4326")
    gdf = gdf.to_crs(4326)[[lake_id_col, "geometry"]].dropna()

    out_rows = []
    for fp in sorted(Path(monthly_dir).glob("S3B_OLCI_EFRNT.*.L3m.MO.*.nc")):
        df_one = extract_lakes_from_nc(str(fp), gdf, lake_id_col, product="monthly")
        out_rows.append(df_one)

    if not out_rows:
        print("No monthly files found.")
        return

    df_all = pd.concat(out_rows, ignore_index=True)
    Path(out_parquet).parent.mkdir(parents=True, exist_ok=True)
    df_all.to_parquet(out_parquet, index=False)
    print(f"[monthly] saved → {out_parquet}  ({len(df_all)} rows)")

run_monthly(
    monthly_dir="/dkucc/home/zy166/HAB-forecasting/datasets/ILW/S3B/2024/CONUS_MO",
    lakes_fp="/dkucc/home/zy166/HAB-forecasting/datasets/Lakes/shapes/lakes_greatlakes_5poly.gpkg",
    lake_id_col="lake_id",
    out_parquet="/dkucc/home/zy166/HAB-forecasting/datasets/processed/lake_ci_monthly.parquet"
)

[monthly] saved → /dkucc/home/zy166/HAB-forecasting/datasets/processed/lake_ci_monthly.parquet  (60 rows)


##### Daily

In [2]:
from pathlib import Path
import geopandas as gpd

def run_daily(daily_dir: str,
              lakes_fp: str,
              lake_id_col: str,
              out_parquet: str):
    """
    daily_dir: Directory containing files like S3M_OLCI_EFRNT.*.L3m.DAY.ILW_CONUS...nc
    """
    daily_dir = Path(daily_dir)
    gdf = gpd.read_file(lakes_fp)
    if gdf.crs is None:
        raise ValueError("The lake file is missing CRS, please ensure it is EPSG:4326")
    gdf = gdf.to_crs(4326)[[lake_id_col, "geometry"]].dropna()

    out_rows = []
    files = sorted(daily_dir.glob("S3M_OLCI_EFRNT.*.L3m.DAY.*.nc"))
    if not files:
        print(f"[WARN] No daily files found under: {daily_dir}")
        return

    for fp in files:
        # 1) 文件头快速校验
        if not looks_like_hdf5(fp):
            print(f"[WARN] Skip (not HDF5/NetCDF header): {fp.name}")
            continue

        # 2) 首选：用你已有的函数（netcdf4 引擎打开）
        try:
            df_one = extract_lakes_from_nc(str(fp), gdf, lake_id_col, product="daily")
            # 统一列名为 date
            if "time" in df_one.columns and "date" not in df_one.columns:
                df_one = df_one.rename(columns={"time": "date"})
            df_one["src"] = fp.name
            df_one["engine"] = "netcdf4"
            out_rows.append(df_one)
            continue
        except Exception as e1:
            print(f"[WARN] netcdf4 failed on {fp.name}: {e1}")

        # 3) 兜底：h5netcdf 内联提取（若可用）
        try:
            df_one = _extract_one_with_h5netcdf(fp, gdf, lake_id_col, product="daily")
            out_rows.append(df_one)
        except Exception as e2:
            print(f"[SKIP] {fp.name}: h5netcdf fallback failed → {e2}")

    if not out_rows:
        print("No valid daily rows produced.")
        return

    df_all = pd.concat(out_rows, ignore_index=True)

    # 基本整理：日期排序、列顺序、小型诊断
    keep_cols = ["lake_id", "date", "product", "CI_mean", "CI_p90", "n_valid", "src", "engine"]
    for c in keep_cols:
        if c not in df_all.columns:
            df_all[c] = np.nan if c not in ("lake_id","product","src","engine") else None
    df_all = df_all[keep_cols].sort_values(["lake_id", "date"]).reset_index(drop=True)

    Path(out_parquet).parent.mkdir(parents=True, exist_ok=True)
    df_all.to_parquet(out_parquet, index=False)
    print(f"[daily] saved → {out_parquet}  (rows={len(df_all)}, files={len(files)})")

run_daily(
    daily_dir="/dkucc/home/zy166/HAB-forecasting/datasets/ILW/Merged/2024/CONUS_DAY",
    lakes_fp="/dkucc/home/zy166/HAB-forecasting/datasets/Lakes/shapes/lakes_greatlakes_5poly.gpkg",
    lake_id_col="lake_id",
    out_parquet="/dkucc/home/zy166/HAB-forecasting/datasets/processed/lake_ci_daily.parquet"
)

[WARN] netcdf4 failed on S3M_OLCI_EFRNT.20240211.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc: [Errno -101] NetCDF: HDF error: '/dkucc/home/zy166/HAB-forecasting/datasets/ILW/Merged/2024/CONUS_DAY/S3M_OLCI_EFRNT.20240211.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc'
[SKIP] S3M_OLCI_EFRNT.20240211.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc: h5netcdf fallback failed → Unable to synchronously open file (truncated file: eof = 323158016, sblock->base_addr = 0, stored_eof = 763662772)
[WARN] netcdf4 failed on S3M_OLCI_EFRNT.20241016.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc: [Errno -101] NetCDF: HDF error: '/dkucc/home/zy166/HAB-forecasting/datasets/ILW/Merged/2024/CONUS_DAY/S3M_OLCI_EFRNT.20241016.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc'
[SKIP] S3M_OLCI_EFRNT.20241016.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc: h5netcdf fallback failed → Unable to synchronously open file (truncated file: eof = 411697152, sblock->base_addr = 0, stored_eof = 978029795)
[daily] saved → /dkucc/home/zy166/HAB-forecasting/datasets/proce

### Data Quality Control

In [3]:
import pandas as pd
import numpy as np

df = pd.read_parquet("/dkucc/home/zy166/HAB-forecasting/datasets/processed/qc/greatlakes_monthly_clean.parquet")
print("Columns:", list(df.columns))
print("Shape:", df.shape)
print(df.head())

for c in ["CI_mean", "CI_p90", "n_valid"]:
    if c in df.columns:
        v = pd.to_numeric(df[c], errors="coerce")
        print(f"{c}: non-null={v.notna().sum()}, finite={np.isfinite(v).sum()}, mean={v.mean() if np.isfinite(v).any() else np.nan}")

Columns: ['lake_id', 'product', 'CI_mean', 'CI_p90', 'n_valid', 'src', 'date', 'area_m2', 'expected_pixels_geom', 'lake_name', 'empiric_n_valid_max', 'pct_valid_geom', 'pct_valid_emp', 'qc_low_cov_geom', 'qc_low_cov_emp', 'qc_tiny_abs_pix', 'qc_is_valid']
Shape: (60, 17)
  lake_id  product  CI_mean  CI_p90  n_valid  \
0    GL-1  monthly      NaN     NaN      NaN   
1    GL-2  monthly      NaN     NaN      NaN   
2    GL-3  monthly      NaN     NaN      NaN   
3    GL-4  monthly      NaN     NaN      NaN   
4    GL-5  monthly      NaN     NaN      NaN   

                                                 src                    date  \
0  S3B_OLCI_EFRNT.20240101_20240131.L3m.MO.ILW_CO... 2024-01-16 16:53:28.500   
1  S3B_OLCI_EFRNT.20240101_20240131.L3m.MO.ILW_CO... 2024-01-16 16:53:28.500   
2  S3B_OLCI_EFRNT.20240101_20240131.L3m.MO.ILW_CO... 2024-01-16 16:53:28.500   
3  S3B_OLCI_EFRNT.20240101_20240131.L3m.MO.ILW_CO... 2024-01-16 16:53:28.500   
4  S3B_OLCI_EFRNT.20240101_20240131.L3m

In [4]:
"""
QC & Completeness checks for NASA ILW Cyanobacteria Index (CI_cyano)
- Regional (CONUS) daily & monthly time series
- Lake-level (Great Lakes) daily & monthly time series

Inputs (already prepared by you):
1) /dkucc/home/zy166/HAB-forecasting/datasets/ILW/Merged/2024/CONUS_DAY/ci_cyano_daily_mean.csv
2) /dkucc/home/zy166/HAB-forecasting/datasets/ILW/S3B/2024/CONUS_MO/ci_cyano_monthly_mean.csv
3) /dkucc/home/zy166/HAB-forecasting/datasets/Lakes/shapes/lakes_greatlakes_5poly.gpkg
4) /dkucc/home/zy166/HAB-forecasting/datasets/processed/lake_ci_monthly.parquet
5) /dkucc/home/zy166/HAB-forecasting/datasets/processed/lake_ci_daily.parquet

Outputs:
- Cleaned regional daily/monthly CSVs with QC flags
- Cleaned lake-level daily/monthly Parquet with QC flags and completeness metrics
- Simple summary CSVs per product (row counts, missing rates, clipping, etc.)
"""

from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd

# ----------------------------
# Parameters (tune as needed)
# ----------------------------
# Resolution of ILW L3m grid (meters)
PIX_RES_M = 300.0

# Minimal valid pixel ratio when judged by geometry (n_valid / expected_pixels)
MIN_PCT_VALID_GEOM = 0.10   # drop if coverage < 10%

# Minimal absolute valid pixels per lake-time
MIN_ABS_PIX = 50            # drop very tiny coverage

# Empirical coverage threshold relative to best observed coverage for that lake
MIN_PCT_VALID_EMP = 0.10    # drop if n_valid < 10% of lake's max observed n_valid

# CI cleaning thresholds
NEAR_ZERO_THRESHOLD = 5e-5  # drop near-zero values (already applied in earlier step, kept as doc)
CLIP_QUANTILE_LOW  = 0.001  # lower clip for outliers
CLIP_QUANTILE_HIGH = 0.999  # upper clip for outliers

# Interpolation limits for regional (optional smoothing to fill short gaps)
INTERP_LIMIT_DAYS = 3

# ----------------------------
# Paths
# ----------------------------
P_CONUS_DAILY   = Path("/dkucc/home/zy166/HAB-forecasting/datasets/ILW/Merged/2024/CONUS_DAY/ci_cyano_daily_mean.csv")
P_CONUS_MONTHLY = Path("/dkucc/home/zy166/HAB-forecasting/datasets/ILW/S3B/2024/CONUS_MO/ci_cyano_monthly_mean.csv")
P_LAKES_GPKG    = Path("/dkucc/home/zy166/HAB-forecasting/datasets/Lakes/shapes/lakes_greatlakes_5poly.gpkg")
P_LAKE_DAILY    = Path("/dkucc/home/zy166/HAB-forecasting/datasets/processed/lake_ci_daily.parquet")
P_LAKE_MONTHLY  = Path("/dkucc/home/zy166/HAB-forecasting/datasets/processed/lake_ci_monthly.parquet")

OUT_DIR = Path("/dkucc/home/zy166/HAB-forecasting/datasets/processed/qc")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ----------------------------
# Utilities
# ----------------------------
def _clip_series_q(s: pd.Series, qlow=CLIP_QUANTILE_LOW, qhigh=CLIP_QUANTILE_HIGH) -> pd.Series:
    """Clip a numeric series by quantiles; preserve NaNs."""
    if s.dropna().empty:
        return s
    lo = s.quantile(qlow)
    hi = s.quantile(qhigh)
    return s.clip(lower=lo, upper=hi)

def _ensure_datetime(df: pd.DataFrame,
                     prefer_cols=("date", "time")) -> pd.DataFrame:
    """
    Robust datetime parsing:
    - try 'date' then 'time' (or any column that contains these names)
    - tolerant to mixed formats; coerce to UTC tz-aware then drop tz
    - drop NaT rows
    """
    col = None
    for cand in prefer_cols:
        if cand in df.columns:
            col = cand
            break
    if col is None:
        for c in df.columns:
            lc = str(c).lower()
            if "date" in lc or "time" in lc or "datetime" in lc or "timestamp" in lc:
                col = c
                break
    if col is None:
        raise ValueError("No recognizable datetime column (expect 'date'/'time').")

    dt = pd.to_datetime(df[col], utc=True, format="mixed", errors="coerce")
    dt = dt.dt.tz_convert("UTC").dt.tz_localize(None)

    df = df.copy()
    df["date"] = dt
    before = len(df)
    df = df.dropna(subset=["date"]).reset_index(drop=True)
    if len(df) < before:
        print(f"[QC] Dropped {before - len(df)} rows with unparseable datetime in column '{col}'")
    return df

def _summarize_basic(df: pd.DataFrame, tag: str) -> pd.DataFrame:
    """Basic counts and missingness summary for quick logging."""
    out = {
        "tag": tag,
        "rows": len(df),
        "n_missing_CI_mean": int(df["CI_mean"].isna().sum()) if "CI_mean" in df.columns else None,
        "n_missing_CI_p90":  int(df["CI_p90"].isna().sum()) if "CI_p90" in df.columns else None,
        "n_missing_n_valid": int(df["n_valid"].isna().sum()) if "n_valid" in df.columns else None,
    }
    return pd.DataFrame([out])

# ----------------------------
# QC for regional (CONUS) time series
# ----------------------------
def qc_conus_timeseries(csv_path: Path, freq: str, out_prefix: str) -> None:
    df = pd.read_csv(csv_path)

    expected = {"CI_mean", "CI_p90", "n_valid"}
    missing = expected - set(df.columns)
    if missing:
        print(f"[WARN] {csv_path.name} is missing columns: {missing}")

    # robust parse + drop NaT
    df = _ensure_datetime(df, prefer_cols=("date", "time"))

    # normalize timestamps to start-of-period
    if freq.upper() == "D":
        df["date"] = df["date"].dt.floor("D")
    elif freq.upper() == "MS":
        df["date"] = df["date"].dt.to_period("M").dt.to_timestamp(how="start")
    else:
        df["date"] = df["date"].dt.to_period(freq).dt.start_time

    df = df.sort_values("date").drop_duplicates(subset=["date"]).reset_index(drop=True)

    # build regular index over the observed span
    idx = pd.date_range(df["date"].min(), df["date"].max(), freq=freq)
    df = df.set_index("date").reindex(idx).rename_axis("date").reset_index()

    # short-gap interpolation on CI columns (both directions)
    for c in ["CI_mean", "CI_p90"]:
        if c in df.columns:
            df[c] = df[c].interpolate(
                limit=INTERP_LIMIT_DAYS if freq.upper()=="D" else 1,
                limit_direction="both"
            )

    # robust quantile clipping
    for c in ["CI_mean", "CI_p90"]:
        if c in df.columns:
            df[c] = _clip_series_q(df[c])

    if "n_valid" in df.columns:
        nv_thr = df["n_valid"].quantile(0.05) if df["n_valid"].notna().any() else 0
        df["qc_low_coverage"] = (df["n_valid"] < nv_thr).astype(int)

    out_csv = OUT_DIR / f"{out_prefix}_clean.csv"
    df.to_csv(out_csv, index=False)
    _summarize_basic(df, tag=f"{out_prefix}").to_csv(OUT_DIR / f"{out_prefix}_summary.csv", index=False)
    print(f"[OK] Saved regional QC → {out_csv}")

# ----------------------------
# Lake geometry → expected pixel count
# ----------------------------
def compute_expected_pixels_per_lake(lakes_gpkg: Path,
                                     inset_m: float = 0.0) -> pd.DataFrame:
    """
    Compute expected pixel counts per lake from geometry area, after optional inward buffer
    (set inset_m=0 to avoid double-shrinking if polygons are already buffered).
    Done in equal-area CRS (EPSG:5070) then back to 4326.

    Returns a DataFrame: lake_id, lake_name (if present), expected_pixels_geom, area_m2
    """
    g = gpd.read_file(lakes_gpkg)
    cols = list(g.columns)
    if "lake_id" not in cols:
        raise ValueError("Expect 'lake_id' in lakes_greatlakes_5poly.gpkg")

    name_col = None
    for cand in ["lake_name", "Lake_name", "name", "Name"]:
        if cand in cols:
            name_col = cand
            break

    gm = g.to_crs(5070)

    if inset_m and inset_m > 0:
        gm["geometry"] = gm.buffer(-abs(inset_m))

    gm["area_m2"] = gm.geometry.area
    gm["expected_pixels_geom"] = (gm["area_m2"] / (PIX_RES_M ** 2)).round().astype("Int64")

    df = gm.to_crs(4326)[["lake_id", "area_m2", "expected_pixels_geom"]].copy()
    if name_col:
        df[name_col] = g[name_col].values

    return df

# ----------------------------
# QC for lake-level time series
# ----------------------------
def _safe_nanmedian(arr) -> float:
    """Return NaN if all values are NaN or array empty; otherwise nanmedian."""
    a = pd.Series(arr).astype(float)
    a = a[np.isfinite(a)]
    return float(np.nanmedian(a)) if len(a) else float("nan")

def _coerce_n_valid(df: pd.DataFrame) -> pd.DataFrame:
    # 1) 如果已有 n_valid 且存在非零值，直接返回
    if "n_valid" in df.columns and (df["n_valid"].fillna(0) > 0).any():
        return df

    # 2) 常见别名兜底
    for alias in ["valid_count", "count_valid", "num_valid", "n", "count"]:
        if alias in df.columns and (df[alias].fillna(0) > 0).any():
            df = df.rename(columns={alias: "n_valid"})
            return df

    # 3) 若 n_valid 不存在或全为 0 → 改成 NaN，避免误触发覆盖率判定
    if "n_valid" not in df.columns or not (df["n_valid"].fillna(0) > 0).any():
        df["n_valid"] = np.nan
        print("[WARN] n_valid is missing or all zeros in lake parquet; "
              "set n_valid = NaN so coverage flags become NA.")
    return df

def qc_lake_timeseries(parquet_path: Path, lakes_gpkg: Path, out_prefix: str) -> None:
    """
    QC for lake-level CI time series (daily or monthly parquet).
    Adds:
      - expected_pixels based on lake geometry
      - pct_valid_geom = n_valid / expected_pixels
      - pct_valid_emp = n_valid / max(n_valid) per lake
      - QC flags: low coverage (geom/emp), tiny absolute pixels
    Outputs cleaned Parquet + summary CSV.
    """
    # Load data
    df = pd.read_parquet(parquet_path)
    df = _coerce_n_valid(df)

    # Robust datetime parse: prefer 'date', then 'time'
    df = _ensure_datetime(df, prefer_cols=("date", "time"))

    # Ensure single datetime column named 'date'
    if "time" in df.columns and "date" in df.columns:
        df = df.drop(columns=["time"])
    elif "time" in df.columns and "date" not in df.columns:
        df = df.rename(columns={"time": "date"})

    # Drop duplicated columns if any
    if df.columns.duplicated().any():
        dup = df.columns[df.columns.duplicated()].tolist()
        print(f"[QC] Dropping duplicated columns: {dup}")
        df = df.loc[:, ~df.columns.duplicated()].copy()

    # Ensure a 'product' column exists
    if "product" not in df.columns:
        df["product"] = "unknown"

    # Expected pixels per lake from geometry (no extra inset here)
    geom_df = compute_expected_pixels_per_lake(lakes_gpkg, inset_m=0.0)
    name_col = "lake_name" if "lake_name" in geom_df.columns else ("Lake_name" if "Lake_name" in geom_df.columns else None)

    # Merge expected pixels
    df = df.merge(geom_df, on="lake_id", how="left")

    # Empirical max n_valid per lake (skip zeros/NaNs)
    if "n_valid" in df.columns:
        emp_max = (
            df.loc[df["n_valid"].fillna(0) > 0]
              .groupby("lake_id")["n_valid"]
              .max()
              .rename("empiric_n_valid_max")
        )
        df = df.merge(emp_max, on="lake_id", how="left")

        # Coverage metrics
        df["pct_valid_geom"] = df["n_valid"] / df["expected_pixels_geom"]
        df["pct_valid_emp"]  = df["n_valid"] / df["empiric_n_valid_max"]

        for c in ("pct_valid_geom", "pct_valid_emp"):
            df.loc[~np.isfinite(df[c]), c] = np.nan

        # Coverage flags
        df["qc_low_cov_geom"] = pd.Series(pd.NA, index=df.index, dtype="Int64")
        ok_geom = df["pct_valid_geom"].notna()
        df.loc[ok_geom, "qc_low_cov_geom"] = (df.loc[ok_geom, "pct_valid_geom"] < MIN_PCT_VALID_GEOM).astype("Int64")

        df["qc_low_cov_emp"] = pd.Series(pd.NA, index=df.index, dtype="Int64")
        ok_emp = df["pct_valid_emp"].notna()
        df.loc[ok_emp, "qc_low_cov_emp"] = (df.loc[ok_emp, "pct_valid_emp"] < MIN_PCT_VALID_EMP).astype("Int64")

        df["qc_tiny_abs_pix"] = (pd.to_numeric(df["n_valid"], errors="coerce") < MIN_ABS_PIX).astype("Int64")
    else:
        for c in ["pct_valid_geom", "pct_valid_emp", "qc_low_cov_geom", "qc_low_cov_emp", "qc_tiny_abs_pix"]:
            df[c] = pd.NA

    # Clip CI columns lake-wise (robust to per-lake distributions)
    for c in ["CI_mean", "CI_p90"]:
        if c in df.columns:
            df[c] = df.groupby("lake_id", group_keys=False)[c].apply(
                lambda s: _clip_series_q(s, CLIP_QUANTILE_LOW, CLIP_QUANTILE_HIGH)
            )

    # Consolidated validity flag
    cov_flags = ["qc_low_cov_geom", "qc_low_cov_emp", "qc_tiny_abs_pix"]
    for c in cov_flags:
        if c not in df.columns:
            df[c] = pd.NA

    df["qc_is_valid"] = 1  # start as valid
    for c in cov_flags:
        df.loc[df[c] == 1, "qc_is_valid"] = 0

    # rows with no coverage info → fall back to "CI_mean is finite"
    rows_no_cov = df[cov_flags].isna().all(axis=1)
    ci_ok = pd.to_numeric(df.get("CI_mean"), errors="coerce")
    ci_ok = ci_ok.apply(lambda x: int(np.isfinite(x)))
    df.loc[rows_no_cov, "qc_is_valid"] = ci_ok.loc[rows_no_cov].values

    # Optional quick diagnostics (helps when a lake looks empty in plots)
    diag = (
        df.assign(CI_mean_finite = pd.to_numeric(df["CI_mean"], errors="coerce").apply(np.isfinite))
          .groupby("lake_id")
          .agg(n_rows=("lake_id","size"),
               n_CI_finite=("CI_mean_finite","sum"),
               n_valid_pos=("n_valid", lambda s: int((pd.to_numeric(s, errors="coerce")>0).sum())),
               qc_valid=("qc_is_valid","sum"))
          .reset_index()
    )
    print("\n[DIAG] per-lake counts:\n", diag.to_string(index=False))

    # Save cleaned parquet
    out_pq = OUT_DIR / f"{out_prefix}_clean.parquet"
    df.to_parquet(out_pq, index=False)

    # Build simple summary per-lake
    summary_rows = []
    for lid, g in df.groupby("lake_id"):
        n_rows = len(g)
        n_valid_rows = int((g["qc_is_valid"] == 1).sum())
        frac_valid = n_valid_rows / n_rows if n_rows else 0.0
        row = {
            "lake_id": lid,
            "rows": n_rows,
            "rows_valid": n_valid_rows,
            "frac_valid": round(frac_valid, 4),
            "pct_valid_geom_med": _safe_nanmedian(g["pct_valid_geom"]) if "pct_valid_geom" in g.columns else np.nan,
            "pct_valid_emp_med":  _safe_nanmedian(g["pct_valid_emp"])  if "pct_valid_emp"  in g.columns else np.nan,
        }
        if name_col and name_col in g.columns:
            row["lake_name"] = g[name_col].iloc[0]
        summary_rows.append(row)

    summary_df = pd.DataFrame(summary_rows).sort_values("lake_id")
    out_summary = OUT_DIR / f"{out_prefix}_summary.csv"
    summary_df.to_csv(out_summary, index=False)

    print(f"[OK] Saved lake-level QC → {out_pq}")
    print(f"[OK] Summary → {out_summary}")

# ----------------------------
# Main
# ----------------------------
if __name__ == "__main__":
    # 1) Regional time series QC
    if P_CONUS_DAILY.exists():
        qc_conus_timeseries(P_CONUS_DAILY,   freq="D",  out_prefix="conus_daily")
    else:
        print(f"[SKIP] Missing: {P_CONUS_DAILY}")

    if P_CONUS_MONTHLY.exists():
        qc_conus_timeseries(P_CONUS_MONTHLY, freq="MS", out_prefix="conus_monthly")
    else:
        print(f"[SKIP] Missing: {P_CONUS_MONTHLY}")

    # 2) Lake-level QC (Great Lakes)
    if P_LAKE_DAILY.exists():
        qc_lake_timeseries(P_LAKE_DAILY,  P_LAKES_GPKG, out_prefix="greatlakes_daily")
    else:
        print(f"[SKIP] Missing: {P_LAKE_DAILY}")

    if P_LAKE_MONTHLY.exists():
        qc_lake_timeseries(P_LAKE_MONTHLY, P_LAKES_GPKG, out_prefix="greatlakes_monthly")
    else:
        print(f"[SKIP] Missing: {P_LAKE_MONTHLY}")

[OK] Saved regional QC → /dkucc/home/zy166/HAB-forecasting/datasets/processed/qc/conus_daily_clean.csv
[OK] Saved regional QC → /dkucc/home/zy166/HAB-forecasting/datasets/processed/qc/conus_monthly_clean.csv
[WARN] n_valid is missing or all zeros in lake parquet; set n_valid = NaN so coverage flags become NA.

[DIAG] per-lake counts:
 lake_id  n_rows  n_CI_finite  n_valid_pos  qc_valid
   GL-1     361            0            0       361
   GL-2     361            0            0       361
   GL-3     361            0            0       361
   GL-4     361            0            0       361
   GL-5     361            0            0       361
[OK] Saved lake-level QC → /dkucc/home/zy166/HAB-forecasting/datasets/processed/qc/greatlakes_daily_clean.parquet
[OK] Summary → /dkucc/home/zy166/HAB-forecasting/datasets/processed/qc/greatlakes_daily_summary.csv
[WARN] n_valid is missing or all zeros in lake parquet; set n_valid = NaN so coverage flags become NA.

[DIAG] per-lake counts:
 lake_id 

### Data Visualization

Available at independent python file `HAB-forecasting/data_visualization_ILW.py`.

## Dataset: Daymet v4 Daily Surface Weather Data (ORNL DAAC)

Read Great Lakes Polygon data (GPKG)

In [ ]:
import geopandas as gpd
import pydaymet as daymet
import xarray as xr
import pandas as pd
from pathlib import Path

GPKG = "/dkucc/home/zy166/HAB-forecasting/datasets/Lakes/shapes/lakes_greatlakes_5poly.gpkg"

lakes = gpd.read_file(GPKG)
# 确保是经纬度坐标（EPSG:4326）；PyDaymet也支持其它CRS，传参loc_crs即可
lakes = lakes.to_crs(4326)

# 准备日期与变量清单
# DATES = ("2016-01-01", "2025-12-31")
DATES = ("2024-01-01", "2024-12-31")
VARS  = ["tmin","tmax","prcp","srad","vp","dayl"]

# 如果没有湖名列，就造一个 id
if "name" not in lakes.columns:
    lakes["name"] = [f"lake_{i}" for i in range(len(lakes))]

逐湖请求 Daymet 网格并做“湖面平均”

In [ ]:
out_dir = Path("/dkucc/home/zy166/HAB-forecasting/datasets/Daymet/glakes_nc")
out_dir.mkdir(parents=True, exist_ok=True)

dfs_daily = []   # 存各湖“日尺度湖面平均”的表

for idx, row in lakes.iterrows():
    geom   = row.geometry
    lakeid = row["name"]

    # 取 Daymet 栅格（daily）
    ds = daymet.get_bygeom(
        geom, DATES, variables=VARS,  # time_scale 默认 daily
        # 如你的 GPKG 不是4326，可加 loc_crs=你的EPSG
    )

    # 按空间维度(y,x)做湖面平均；skipna 处理缺测
    ds_mean = ds[VARS].mean(dim=["y","x"], skipna=True)

    # 存 netCDF 以便复用
    ds.to_netcdf(out_dir / f"daymet_{lakeid}_daily.nc")

    # 转成 DataFrame（带时间索引）
    df = ds_mean.to_dataframe().reset_index()
    df = df.rename(columns={"time": "date"})
    df["lake_id"] = lakeid
    dfs_daily.append(df)

daymet_daily = pd.concat(dfs_daily, ignore_index=True)

与 ILW 时间步对齐：周均 / 28 天滑动均值

In [ ]:
# 先确保日期是 datetime
daymet_daily["date"] = pd.to_datetime(daymet_daily["date"])

# --- 周聚合（自然周，对齐周一/周日可改 label/closed） ---
wk = (daymet_daily
      .set_index("date")
      .groupby("lake_id")
      [VARS].resample("7D").mean()
      .reset_index()
     )

# --- 28天移动平均（滑窗，不跳步；末端可能因窗口不满而NA） ---
daymet_daily = daymet_daily.sort_values(["lake_id","date"])
roll28 = (daymet_daily
          .groupby("lake_id")[VARS]
          .rolling("28D", on=daymet_daily["date"])
          .mean()
          .reset_index()
         )
roll28 = roll28.rename(columns={"level_1":"date"})